In [ ]:
from pathlib import Path

import pandas as pd

from analytics.app.data.load import list_pipelines, load_pipeline_logs
from analytics.app.data.transform import patch_yearbook_time, pipeline_leaf_times_df
from analytics.plotting.common.cost_matrix import plot_cost_matrix
from analytics.plotting.common.save import save_plot
from modyn.supervisor.internal.pipeline_executor.models import PipelineLogs

%load_ext autoreload
%autoreload 2

In [ ]:
pipelines_dirs = [
    Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/20_datadrift_static"),
]

pipeline_logs: dict[int, PipelineLogs] = {}
pipelines: dict[int, tuple[str, Path]] = {}

for dir in pipelines_dirs:
    dir_pipelines = list_pipelines(dir)
    pipelines.update(dir_pipelines)
    max_pipeline_id = max(dir_pipelines.keys())
    print(pipelines)
    pipeline_logs.update({p_id: load_pipeline_logs(p_id, dir) for (p_id, (_, p_path)) in dir_pipelines.items()})
    assert dir.exists()

In [ ]:
# mode: time + amount
pipeline_ids = [107]  # yb drift mmd 0.06 250 4d

# doesn't do anything unless include_composite_model = True
composite_model_variant = "currently_active_model"

patch_yearbook = True
dataset_id = "yearbook_test"
eval_handler = "periodic-delta+-1y"
metric = "Accuracy"
include_composite_model = True

# Wrangle data

In [ ]:
df_leaf_list = []
for pipeline_id in pipeline_ids:
    logs = pipeline_logs[pipeline_id]
    df_leaf_single = pipeline_leaf_times_df(logs, use_traintime_patch_at_trainer=True, pipeline_id=pipeline_id)
    df_leaf_list.append(df_leaf_single)

df_leaf = pd.concat(df_leaf_list)

In [ ]:
df_adjusted = df_leaf.copy()

# coloring in order of decreasing avg. duration
avg_duration_per_stage = df_adjusted.groupby(["pipeline_ref", "id"])["duration"].mean().sort_values(ascending=False)
df_adjusted = df_adjusted.merge(avg_duration_per_stage, on=["pipeline_ref", "id"], suffixes=("", "_avg")).sort_values(
    "duration_avg", ascending=False
)

# Yearbook as a mapped time dimension (to display the correct timestamps we need to convert back from days to years)
if patch_yearbook:
    patch_yearbook_time(df_adjusted, "sample_time")

In [ ]:
df_adjusted
df_adjusted["sample_time_year"] = df_adjusted["sample_time"].dt.year
df_adjusted["sample_time_year_bin"] = pd.cut(df_adjusted["sample_time_year"], bins=10, labels=False)

In [ ]:
df_new = df_adjusted[
    (
        df_adjusted["id"].isin(
            [
                "TRAIN",
                "STORE_TRAINED_MODEL",
                "INFORM_SELECTOR_REMAINING_DATA",
                "INFORM_SELECTOR_ABOUT_TRIGGER",
                "EVALUATE_TRIGGER_POLICY",
            ]
        )
    )
][["pipeline_ref", "id", "sample_time_year", "duration"]].copy()
df_new = df_new.sort_values("sample_time_year")

In [ ]:
state_rename = {
    "INFORM_SELECTOR_REMAINING_DATA": "inform remaining data",
    "INFORM_SELECTOR_ABOUT_TRIGGER": "inform trigger",
}

df_new["id"] = df_new["id"].replace(state_rename).str.lower().str.replace("_", " ")

In [ ]:
fig = plot_cost_matrix(
    df_new,
    [107],
    grid_alpha=0.75,
    title_map={
        107: "static MMD=0.07 threshold",
    },
    height_factor=0.8,
    width_factor=1.0,
    duration_ylabel="Duration (sec)",
    cumulative_ylabel="Cumulative Duration (min)",
    x_ticks=[x for x in range(1940, 2010 + 1, 30)],
    y_ticks_cumulative=[x for x in range(0, 60 + 1, 20)],
    y_lim_cumulative=(0, 70),
    y_minutes=False,
    y_minutes_cumulative=True,
)

save_plot(fig, "yearbook_drift-trigger-cost-matrix")